Install required libraries

```
!pip install sagemaker
!pip install transformers

```

Importing necessary libraries:


```
import sagemaker
from sagemaker import get_execution_role
from sagemaker.transformer import Transformer
from transformers import AutoTokenizer

import boto3
import pandas as pd

```

Set up SageMaker session and role:



```
sagemaker_session = sagemaker.Session()
role = get_execution_role()

```
Specify the Hugging Face model:


```
model_name = 'cardiovascular-disease-prediction-model'

```
Create an instance of the Hugging Face model:





In [ ]:
hf_model = sagemaker.huggingface.HuggingFaceModel(
    model_data=model_name,
    role=role,
    transformers_version='4.10',
    pytorch_version='1.9',
    py_version='py3',
)

Deploy the model:

In [ ]:
predictor = hf_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
)

Prepare input data:

In [ ]:
data = pd.read_csv('input.csv')
input_text = data['text_column'].tolist()


Tokenize input data:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenized_input = tokenizer(
    input_text,
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors='pt',
)


Invoke model for predictions:

In [ ]:
predictions = predictor.predict(tokenized_input['input_ids'].numpy())


Process predictions:

In [ ]:
predicted_labels = [predicted['label'] for predicted in predictions]
data['predicted_label'] = predicted_labels


Save output:

In [ ]:
data.to_csv('output.csv', index=False)
